In [22]:
import pandas as pd
import numpy as np
import time
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import metrics

In [23]:
vbc = pd.read_csv("../data/vb_data_3_categZone.csv")
vb = pd.read_csv("../data/vb_data_3_numZone.csv")
print(len(vb), 'lines loaded')

146050 lines loaded


In [24]:
X = vb.drop(['Season', 'GameID', 'PlayerTeam', 'PlayerName', 'RewardDistance', 'RewardValue'], axis=1)
cols = [col for col in list(X.columns) if X[col].dtype == 'object']
X_orig = pd.get_dummies(data=X, columns = cols)
Y = vb.RewardValue
print(len(X_orig.columns), 'columns in dataframe')

204 columns in dataframe


In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_orig)
X = scaler.transform(X_orig)

In [26]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(200, input_dim=204, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='tanh'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [27]:
t = time.time()
kfold = KFold(n_splits=5)

for k in range(1,16):
    estimator = KerasRegressor(build_fn=baseline_model, epochs=k, batch_size=64, verbose=0)
    results = cross_val_score(estimator, X, Y, cv=kfold)
    print("CV Result for %d epochs: %.5f (%.2f) MSE" % (k, results.mean(), results.std()))
    
    estimator = KerasRegressor(build_fn=baseline_model, epochs=k, batch_size=64, verbose=0)
    estimator.fit(X,Y)
    yp = estimator.predict(X)
    print("Result for %d epochs: %.5f MSE" % (k, metrics.mean_squared_error(Y,yp)))
    print("Time elapsed:", time.time()-t)
    print()

CV Result for 1 epochs: -0.62570 (0.01) MSE
Result for 1 epochs: 0.61457 MSE
Time elapsed: 9.164573192596436

CV Result for 2 epochs: -0.61484 (0.01) MSE
Result for 2 epochs: 0.60962 MSE
Time elapsed: 23.737534523010254

CV Result for 3 epochs: -0.61441 (0.01) MSE
Result for 3 epochs: 0.61112 MSE
Time elapsed: 44.599096059799194

CV Result for 4 epochs: -0.61618 (0.01) MSE
Result for 4 epochs: 0.61110 MSE
Time elapsed: 70.98557090759277

CV Result for 5 epochs: -0.61448 (0.01) MSE
Result for 5 epochs: 0.60656 MSE
Time elapsed: 102.74872922897339

CV Result for 6 epochs: -0.61391 (0.01) MSE
Result for 6 epochs: 0.60665 MSE
Time elapsed: 138.54797291755676

CV Result for 7 epochs: -0.61156 (0.01) MSE
Result for 7 epochs: 0.60674 MSE
Time elapsed: 180.14660835266113

CV Result for 8 epochs: -0.61469 (0.01) MSE
Result for 8 epochs: 0.60508 MSE
Time elapsed: 227.0126690864563

CV Result for 9 epochs: -0.61424 (0.01) MSE
Result for 9 epochs: 0.60467 MSE
Time elapsed: 279.1561858654022

CV Re

In [28]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=7, batch_size=64, verbose=1)
estimator.fit(X,Y)
yp = estimator.predict(X)
print(metrics.mean_squared_error(Y,yp))

Epoch 1/7
2283/2283 [==============================] - 1s 600us/step - loss: 0.6656
Epoch 2/7
2283/2283 [==============================] - 1s 606us/step - loss: 0.6191
Epoch 3/7
2283/2283 [==============================] - 1s 599us/step - loss: 0.6149
Epoch 4/7
2283/2283 [==============================] - 1s 587us/step - loss: 0.6130
Epoch 5/7
2283/2283 [==============================] - 1s 577us/step - loss: 0.6116
Epoch 6/7
2283/2283 [==============================] - 1s 593us/step - loss: 0.6100
Epoch 7/7
2283/2283 [==============================] - 1s 352us/step
0.6059091693598161


In [29]:
import pickle
pickle.dump(yp, open('nn_regr.pkl', 'wb'))